In [1]:
%jsroot

In [2]:
    // 打开ROOT文件并获取TTree对象
    // TFile *file = TFile::Open("TriggerEvent.root");
    // TFile *file = TFile::Open("1cm_RUN2408_SB12_4h40min_TriggerEvent.root");
    // TFile *file = TFile::Open("TriggerEvent_LD_SB12_T6at9p46mK_bkg_4h40min_5kHz_2408200106.root");
    TFile *file = TFile::Open("/mnt/d/storage/research/RUN2408_data_ana/1cmLMO_SB13_T6at8p5mK_bkg_9h20min_5kHz_2408232128/data/TriggerEvent.root");
    if (!file || file->IsZombie()) {
        std::cerr << "Error: Could not open file TriggerEvent.root" << std::endl;
        return 1;
    }
    TTree *tree = (TTree *)file->Get("tree1");
    if (!tree) {
        std::cerr << "Error: Could not find TTree 'tree1' in the file" << std::endl;
        return 1;
    }

    // tree->Print();

In [3]:
// 定义变量来存储Amp_filtered, RiseTime和DecayTime的值
Float_t Amp_filtered;
Long64_t RiseTime, DecayTime, MaxPos;
Double_t Baseline, Amp_raw;
Int_t nBins_Amp = 500, nBins_Risetime = 80, nBins_Decaytime = 70, nBins_Baseline = 500, nBins_time = 1E3;

// 设置分支的地址
tree->SetBranchAddress("Amp_filtered", &Amp_filtered);
tree->SetBranchAddress("RiseTime", &RiseTime);
tree->SetBranchAddress("DecayTime", &DecayTime);
tree->SetBranchAddress("Baseline", &Baseline);
tree->SetBranchAddress("MaxPos", &MaxPos);
tree->SetBranchAddress("Amp_raw", &Amp_raw);


// 创建画布和直方图
TCanvas *canvas = new TCanvas("canvas", "Amp_filtered", 800, 600);
TH1F *hAmpFiltered = new TH1F("hAmpFiltered", "Filtered AMP Distribution", nBins_Amp, 0, 25000);

TCanvas *canvas2 = new TCanvas("canvas2", "RiseTime", 800, 600);
TH1F *hRiseTime = new TH1F("hRiseTime", "Rise Time Distribution", nBins_Risetime, 0, 80);

TCanvas *canvas3 = new TCanvas("canvas3", "DecayTime", 800, 600);
TH1F *hDecayTime = new TH1F("hDecayTime", "Decay Time Distribution", nBins_Decaytime, 0, 70);

TCanvas *canvas4 = new TCanvas("canvas4", "Baseline", 800, 600);
TH1F *hBaseline = new TH1F("hBaseline", "Baseline Distribution", nBins_Baseline, -14500, -5000);

TCanvas *canvas5 = new TCanvas("canvas5", "BLandAmpOF", 800, 600);
TH2F *hBLandAmpOF = new TH2F("hBLandAmpOF", "Baseline vs Amp_filtered", nBins_Baseline, -14500, -5000, nBins_Amp, 0, 25000);

TCanvas *canvas6 = new TCanvas("canvas6", "BLandAmpRaw", 800, 600);
TH2F *hBLandAmpRaw = new TH2F("hBLandAmpRaw", "Baseline vs Amp_Raw", nBins_Baseline, -14500, -5000, nBins_Amp, 0, 25000);

TCanvas *canvas7 = new TCanvas("canvas7", "BLandMaxPos", 800, 600);
TH2F *hBLandMaxPos = new TH2F("hBLandMaxPos", "Baseline vs time", nBins_time, 0, 8.4E7, nBins_Baseline, -14500, -5000);

TCanvas *canvas8 = new TCanvas("canvas8", "Amp_raw", 800, 600);
TH1F *hAmpRaw = new TH1F("hAmpRaw", "AMP Distribution", nBins_Amp, 0, 25000);

TCanvas *canvas9 = new TCanvas("canvas9", "AmpandDecaytime", 800, 600);
TH2F *hAmpandDecaytime = new TH2F("hAmpandDecaytime", "Amp vs Decaytime", nBins_Amp, 0, 25000, nBins_Decaytime, 0, 70);

TCanvas *canvas10 = new TCanvas("canvas10", "AmpandAmpFiltered", 800, 600);
TH2F *hAmpandAmpFiltered = new TH2F("hAmpandAmpFiltered", "Amp vs AmpFiltered", nBins_Amp, 0, 25000, nBins_Amp, 0, 25000);

// 遍历TTree并填充直方图
Long64_t nentries = tree->GetEntries();
for (Long64_t i = 0; i < nentries; ++i) {
    tree->GetEntry(i);

    hAmpFiltered->Fill(Amp_filtered);
    hRiseTime->Fill(RiseTime);
    hDecayTime->Fill(DecayTime);
    hBaseline->Fill(Baseline);
    hBLandAmpOF->Fill(Baseline, Amp_filtered);
    hBLandAmpRaw->Fill(Baseline, Amp_raw);
    hBLandMaxPos->Fill(MaxPos, Baseline);
    hAmpRaw->Fill(Amp_raw);
    hAmpandDecaytime->Fill(Amp_raw, DecayTime);
    hAmpandAmpFiltered->Fill(Amp_raw, Amp_filtered);
}

// 对 hRiseTime 进行高斯拟合
TF1 *gausFitRiseTime = new TF1("gausFitRiseTime", "gaus", 0, 30);
// TF1 *gausFitRiseTime = new TF1("gausFitRiseTime", "[0]*exp(-0.5*((x-[1])/[2])^2) + [3]", 20, 40);
hRiseTime->Fit(gausFitRiseTime, "R");  // "R" 选项表示限制拟合范围在函数定义的范围内

// 对 hDecayTime 进行高斯拟合
TF1 *gausFitDecayTime = new TF1("gausFitDecayTime", "gaus", 30, 40);
// TF1 *gausFitDecayTime = new TF1("gausFitDecayTime", "[0]*exp(-0.5*((x-[1])/[2])^2) + [3]", 50, 90);
hDecayTime->Fit(gausFitDecayTime, "R");  // "R" 选项表示限制拟合范围在函数定义的范围内

// 设置直方图标题和轴标签
hAmpFiltered->SetTitle("Filtered AMP Distribution");
hAmpFiltered->GetXaxis()->SetTitle("Amp after OF");
hAmpFiltered->GetYaxis()->SetTitle("Counts");
canvas->cd();
gPad->SetLogy();
hAmpFiltered->Draw();

hRiseTime->SetTitle("Rise Time Distribution");
hRiseTime->GetXaxis()->SetTitle("RiseTime (points)");
hRiseTime->GetYaxis()->SetTitle("Counts");
canvas2->cd();
hRiseTime->Draw();
gausFitRiseTime->Draw("same");  // 绘制拟合结果

// 显示 hRiseTime 的拟合参数和统计信息
gStyle->SetOptFit(1);  // 打开拟合参数显示
TPaveStats *statsRiseTime = (TPaveStats*)hRiseTime->GetListOfFunctions()->FindObject("stats");
if (statsRiseTime) {  // 确保 statsRiseTime 不为空
    statsRiseTime->SetX1NDC(0.7); // 设置统计框的位置
    statsRiseTime->SetX2NDC(0.9);
    statsRiseTime->SetY1NDC(0.7);
    statsRiseTime->SetY2NDC(0.9);
}

hDecayTime->SetTitle("Decay Time Distribution");
hDecayTime->GetXaxis()->SetTitle("DecayTime (points)");
hDecayTime->GetYaxis()->SetTitle("Counts");
canvas3->cd();
hDecayTime->Draw();
gausFitDecayTime->Draw("same");  // 绘制拟合结果

// 显示 hDecayTime 的拟合参数和统计信息
TPaveStats *statsDecayTime = (TPaveStats*)hDecayTime->GetListOfFunctions()->FindObject("stats");
if (statsDecayTime) {  // 确保 statsDecayTime 不为空
    statsDecayTime->SetX1NDC(0.7); // 设置统计框的位置
    statsDecayTime->SetX2NDC(0.9);
    statsDecayTime->SetY1NDC(0.7);
    statsDecayTime->SetY2NDC(0.9);
}

hBaseline->SetTitle("Baseline Distribution");
hBaseline->GetXaxis()->SetTitle("Baseline value");
hBaseline->GetYaxis()->SetTitle("Counts");
canvas4->cd();
hBaseline->Draw();

hBLandAmpOF->SetTitle("Baseline vs Filtered Amp");
hBLandAmpOF->GetXaxis()->SetTitle("Baseline value");
hBLandAmpOF->GetYaxis()->SetTitle("Amp after OF");
// canvas5->cd();
// hBLandAmpOF->Draw();
hBLandAmpOF->SetMarkerStyle(20);
hBLandAmpOF->SetMarkerSize(0.8);
canvas5->cd();
gPad->SetLogz();
hBLandAmpOF->Draw("COLZ");

hBLandAmpRaw->SetTitle("Baseline vs Filtered Raw");
hBLandAmpRaw->GetXaxis()->SetTitle("Baseline value");
hBLandAmpRaw->GetYaxis()->SetTitle("Raw Amp");
// canvas6->cd();
// hBLandAmpRaw->Draw();
hBLandAmpRaw->SetMarkerStyle(20);
hBLandAmpRaw->SetMarkerSize(0.8);
canvas6->cd();
gPad->SetLogz();
hBLandAmpRaw->Draw("COLZ");

hBLandMaxPos->SetTitle("Baseline vs Time");
hBLandMaxPos->GetXaxis()->SetTitle("Time");
hBLandMaxPos->GetYaxis()->SetTitle("Baseline");
// canvas7->cd();
// hBLandMaxPos->Draw();
hBLandMaxPos->SetMarkerStyle(20);
hBLandMaxPos->SetMarkerSize(0.8);
canvas7->cd();
gPad->SetLogz();
hBLandMaxPos->Draw("COLZ");

hAmpRaw->SetTitle("AMP Distribution");
hAmpRaw->GetXaxis()->SetTitle("Raw Amp");
hAmpRaw->GetYaxis()->SetTitle("Counts");
canvas8->cd();
gPad->SetLogy();
hAmpRaw->Draw();

hAmpandDecaytime->SetTitle("Amp vs Decaytime");
hAmpandDecaytime->GetXaxis()->SetTitle("Amp");
hAmpandDecaytime->GetYaxis()->SetTitle("Decaytime");
// canvas9->cd();
// hAmpandDecaytime->Draw();
hAmpandDecaytime->SetMarkerStyle(20);
hAmpandDecaytime->SetMarkerSize(0.8);
canvas9->cd();
gPad->SetLogz();
hAmpandDecaytime->Draw("COLZ");

hAmpandAmpFiltered->SetTitle("Amp vs Amp_filtered");
hAmpandAmpFiltered->GetXaxis()->SetTitle("Amp_raw");
hAmpandAmpFiltered->GetYaxis()->SetTitle("Amp_filtered");
hAmpandAmpFiltered->SetMarkerStyle(20);
hAmpandAmpFiltered->SetMarkerSize(0.8);
canvas10->cd();
gPad->SetLogz();
hAmpandAmpFiltered->Draw("COLZ");

 FCN=39381.2 FROM MIGRAD    STATUS=CONVERGED      92 CALLS          93 TOTAL
                     EDM=3.75912e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     3.15266e+04   8.11033e+01   5.52942e+00   1.10461e-05
   2  Mean         1.26060e+01   7.28718e-03   6.79934e-04   5.82183e-02
   3  Sigma        3.86427e+00   7.10128e-03   3.47952e-05   1.83927e+00
 FCN=507.274 FROM MIGRAD    STATUS=CONVERGED     140 CALLS         141 TOTAL
                     EDM=2.58538e-07    STRATEGY= 1  ERROR MATRIX UNCERTAINTY  11.9 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     5.30205e+03   3.52748e+01   9.82691e-01  -2.54398e-05
   2  Mean         3.81113e+01   1.46271e+00   4.66806e-02   1.74688e-04
   3  Sigma        2

In [4]:
canvas->Draw();
canvas2->Draw();
canvas3->Draw();
canvas4->Draw();
canvas5->Draw();
canvas6->Draw();
canvas7->Draw();
canvas8->Draw();
canvas9->Draw();
canvas10->Draw();